In [17]:
import pandas as pd
import re
from collections import Counter

def read_and_rename_columns_with_years(filepath, sep=';', encoding='utf-8'):
    # Kopfzeilen lesen (erste 2 Zeilen)
    df_headers = pd.read_csv(filepath, nrows=2, sep=sep, encoding=encoding, quotechar='"')

    # Erste Zeile: Spaltennamen, zweite Zeile: mögliche Jahreszahlen
    original_headers = df_headers.iloc[0].tolist()
    possible_years = df_headers.iloc[1].tolist()

    renamed_columns = []
    name_counter = Counter()

    # Spaltennamen anpassen
    for col, year in zip(original_headers, possible_years):
        year_str = str(year).strip() if pd.notna(year) else ''
        
        # Wenn es eine Jahreszahl gibt, hänge sie an den Spaltennamen
        if re.match(r'\d{4}', year_str):  # Wenn Jahr im Format YYYY
            new_col_name = f"{col}_{year_str}"
        else:
            new_col_name = col  # Wenn keine Jahreszahl, behalten wir den ursprünglichen Namen

        # Bei Duplikaten im Spaltennamen fügen wir einen Index hinzu
        name_counter[new_col_name] += 1
        if name_counter[new_col_name] > 1:
            new_col_name = f"{new_col_name}_{name_counter[new_col_name] - 1}"

        renamed_columns.append(new_col_name)

    # Ab der dritten Zeile die tatsächlichen Daten einlesen
    df = pd.read_csv(
        filepath,
        skiprows=2,
        names=renamed_columns,
        sep=sep,
        encoding=encoding,
        quotechar='"'
    )

    # Raumeinheit-Spalte löschen (falls vorhanden)
    df.drop(columns=['Raumeinheit'], errors='ignore', inplace=True)

    return df


In [18]:
zensus_csv_path = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/INKAR/Absolutzahlen.csv"
df_zensus = read_and_rename_columns_with_years(zensus_csv_path)
print(df_zensus[['Kennziffer']].head(10))  # Ausgabe der ersten 10 Zeilen der "Kennziffer"-Spalte


KeyError: "None of [Index(['Kennziffer'], dtype='object')] are in the [columns]"

In [19]:
# Überprüfe alle Spaltennamen
print(df_zensus.columns)


Index(['nan_1001.0',          nan,      'nan_1',       '2022',     '2022_1',
           '2022_2',     '2022_3',     '2022_4',     '2022_5',       '2021',
           '2022_6',     '2022_7',     '2022_8',     '2021_1'],
      dtype='object')
